In [45]:
import plotly.express as px
import panel as pn
import pandas as pd
import os
from pathlib import Path
import pprint
import backtrader as bt
from bokeh.plotting import figure, output_file, show
import bokeh
import hvplot.pandas
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [2]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [3]:
def btc_plot():
    file_path = Path("Data/BTCUSDT.csv")
    btc_data = pd.read_csv(file_path, header = None)
    btc_data.columns = ["Date", "Open", "High", "Low", "Close", "Volume", "Open Interst"]
    btc_data.head()

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                   vertical_spacing=0.3, subplot_titles=('BTC Price Data', 'Volume'), 
                   row_width=[0.2, 0.7])

    fig.add_trace(go.Candlestick(
        x = btc_data["Date"],
        open = btc_data["Open"],
        high = btc_data["High"],
        low = btc_data["Low"],
        close = btc_data["Close"],
        name="OHLC", 
        showlegend=False), 
        row=1, 
      col=1
    )
    fig.add_trace(go.Bar(
        x = btc_data["Date"],
        y = btc_data["Volume"],
        showlegend=False), 
        row=2, 
        col=1)
    return fig


In [4]:
# Create a simple strategy which extends off the backtrader strategy class
# This strategy is a simple moving average cross, which takes buy orders when the 10 period MA crosses the 50 period MA
class MaCrossStrategy(bt.Strategy):
    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)
         
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)
 
    def next(self):
        if not self.position:
            if self.crossover > 0: 
                self.buy()
        elif self.crossover < 0: 
            self.close()

In [5]:
# This function inputs csv data and runs the MA cross strategy across it.
# It returns an object which will allow for analysis
# Functions that can be called on this object include 
def backtrader_run(file_path):
#     file_path = Path("data/BTCUSDT.csv")
    # This sets up the engine which backtests the strategy
    cerebro = bt.Cerebro(maxcpus = None)
    cerebro.broker.setcash(1000.0)
    data = bt.feeds.GenericCSVData(dataname=file_path, timeframe=bt.TimeFrame.Days, openinterest=-1)
    cerebro.adddata(data)
    cerebro.addstrategy(MaCrossStrategy)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=50)
    
    # Add analyzers so data can be used for analysis and visualizations
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name = "SharpeRatio")
    cerebro.addanalyzer(bt.analyzers.Transactions, _name = "Transactions")
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer,_name = "TradeAnalyzer")
    cerebro.addanalyzer(bt.analyzers.DrawDown,_name = "DrawDown")
    cerebro.addanalyzer(bt.analyzers.Returns,_name = "Returns")
    
    result = cerebro.run()
    
    return result

In [6]:
btc_results = backtrader_run("data/BTCUSDT.csv")
# pprint.pprint(btc_results[0].analyzers.SharpeRatio.get_analysis())
# pprint.pprint(btc_results[0].analyzers.Transactions.get_analysis())
# pprint.pprint(btc_results[0].analyzers.TradeAnalyzer.get_analysis())
# pprint.pprint(btc_results[0].analyzers.DrawDown.get_analysis())
# pprint.pprint(btc_results[0].analyzers.Returns.get_analysis())

In [7]:
sp500_results = backtrader_run("SP500_Data1.csv")
# pprint.pprint(sp500_results[0].analyzers.SharpeRatio.get_analysis())
# pprint.pprint(sp500_results[0].analyzers.Transactions.get_analysis())
# pprint.pprint(sp500_results[0].analyzers.TradeAnalyzer.get_analysis())
# pprint.pprint(sp500_results[0].analyzers.DrawDown.get_analysis())
# pprint.pprint(sp500_results[0].analyzers.Returns.get_analysis())

In [8]:
btc_transactions = pd.DataFrame()
btc_pnl = pd.DataFrame()
index = 0

for date, transaction in btc_results[0].analyzers.Transactions.get_analysis().items():
    btc_transactions = btc_transactions.append([{"Date" : date, "Transaction" : transaction[0][4]}], ignore_index=True)
    if index%2 ==1:
        btc_pnl = btc_pnl.append({"Date" : date, "Closed PnL" :btc_transactions.iloc[index-1][1] + btc_transactions.iloc[index][1]}, ignore_index=True)
    index += 1
btc_transactions = btc_transactions.set_index("Date")
btc_pnl = btc_pnl.set_index("Date")

btc_pnl.hvplot.bar(rot = 90)

:Bars   [Date]   (Closed PnL)

In [9]:
def plot_pnl(title, results):
    transactions = pd.DataFrame()
    pnl = pd.DataFrame()
    index = 0

    for date, transaction in results[0].analyzers.Transactions.get_analysis().items():
        transactions = transactions.append([{"Date" : date, "Transaction" : transaction[0][4]}], ignore_index=True)
        if index%2 ==1:
            pnl = pnl.append({"Date" : date, "Closed PnL" :transactions.iloc[index-1][1] + transactions.iloc[index][1]}, ignore_index=True)
        index += 1
    transactions = transactions.set_index("Date")
    pnl = pnl.set_index("Date")

    return pnl.hvplot.bar(title = title, rot = 90)

In [10]:
plot_pnl("BTC Profit and Loss", btc_results)

:Bars   [Date]   (Closed PnL)

In [35]:
def btc_pnl_comparison():
    file_path = Path("Data/BTCUSDT.csv")
    btc_data = pd.read_csv(file_path, header = None)
    # print(btc_data.tail())
    btc_data.iloc[0][4]
    strategy_return = btc_pnl.sum()[0]
    hodl_return = btc_data.iloc[-1][4] - btc_data.iloc[0][4]

    pnl = pd.DataFrame([["Hodl", hodl_return], ["MA Strategy", strategy_return]], columns = ["Strategy", "Return"])
    pnl = pnl.set_index("Strategy")
    
    return pnl.hvplot.bar(title = "BTC PnL Comparison")


In [55]:
def btc_sharpe_ratio():
    file_path = Path("Data/BTCUSDT.csv")
    btc_data = pd.read_csv(file_path, header = None)
    # print(btc_data.head())
    btc_daily_returns = btc_data[4].pct_change()
    btc_daily_returns = btc_daily_returns.dropna()
    hodl_sharpe_ratio = (btc_daily_returns.mean()*252)/(btc_daily_returns.std() * np.sqrt(252))


    sharpe_ratio = pd.DataFrame([["Hodl", hodl_sharpe_ratio], ["MA Strategy", btc_results[0].analyzers.SharpeRatio.get_analysis()['sharperatio']]], columns = ["Strategy", "Sharpe Ratio"])
    sharpe_ratio = sharpe_ratio.set_index("Strategy")
    return sharpe_ratio.hvplot.bar(title = "BTC Sharpe Ratio Comparison")

In [56]:
btc_column = pn.Column(
    "## BTC Plot", btc_plot(),
    '## BTC Moving Average Strategy PnL', plot_pnl("BTC Profit and Loss", btc_results),
    '## BTC PnL Comparison', btc_pnl_comparison(),
    '## BTC Sharpe Ratio Comparison', btc_sharpe_ratio()
)

strategy_dashboard = pn.Tabs(
    ("BTC", btc_column)
)

In [57]:
strategy_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
        [2] Markdown(str)
        [3] HoloViews(Bars)
        [4] Markdown(str)
        [5] HoloViews(Bars)
        [6] Markdown(str)
        [7] HoloViews(Bars)

In [ ]:
# show(bokeh.mpl.to_bokeh(backtrader_plot))

In [ ]:
# bokeh.mpl.to_bokeh(backtrader_plot)